<a href="https://www.kaggle.com/code/alawdisoft/arabic-sentmental-anlysis?scriptVersionId=134253798" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

* Import library


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import os

* read data

In [2]:
mainpath ='/kaggle/input/arabic-sentiment-twitter-corpus'
train_neg = pd.read_csv(mainpath+'/test_Arabic_tweets_negative_20190413.tsv'  , sep='\t' , header=None) 
train_pos = pd.read_csv(mainpath+'/train_Arabic_tweets_positive_20190413.tsv' , sep='\t', header=None) 
test_neg = pd.read_csv(mainpath+'/test_Arabic_tweets_negative_20190413.tsv' , sep='\t', header=None) 
test_pos = pd.read_csv(mainpath+'/test_Arabic_tweets_positive_20190413.tsv' , sep='\t', header=None) 

In [3]:
train = pd.concat([train_pos, train_neg], axis=0, ignore_index=True, 
                  keys=['positive', 'negative'], names=['sentiment', 'index'])
train.columns = ['target', 'tweets']

test = pd.concat([test_pos, test_neg], axis=0, ignore_index=True, 
                  keys=['positive', 'negative'], names=['sentiment', 'index'])
test.columns = ['target', 'tweets']

* Data cleaning

In [4]:
train.isnull().sum()

target    0
tweets    0
dtype: int64

In [5]:
test.isnull().sum()

target    0
tweets    0
dtype: int64

In [6]:
train.duplicated().sum()

9615

In [7]:
test.duplicated().sum()

2679

In [8]:
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

* Remove Emoji 

In [9]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

train['tweets'] = train['tweets'].apply( lambda s: remove_emojis(s) )
test['tweets']  = test['tweets'].apply( lambda s: remove_emojis(s) )


* Remove punctuation

In [10]:
import string

def remove_punctuation(text):
    translator = str.maketrans(' ', ' ', string.punctuation)
    return text.translate(translator)

train['tweets'] = train['tweets'].apply( lambda s: remove_punctuation(s) )
test['tweets']  = test['tweets'].apply( lambda s: remove_punctuation(s) )

* Remove stops words

In [11]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords_arabic(text):
    stop_words = set(stopwords.words('arabic'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

train['tweets'] = train['tweets'].apply( lambda s: remove_stopwords_arabic(s) )
test['tweets']  = test['tweets'].apply( lambda s: remove_stopwords_arabic(s) )

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
mapping = {'neg':0 , 'pos':1}

train['target'] = train['target'].map(mapping)
test['target'] = test['target'].map(mapping)

In [13]:
cv = CountVectorizer(binary=True)
cv.fit(train['tweets'])
X = cv.transform(train['tweets'] )
X_test = cv.transform(test['tweets'] )

In [14]:
# X_train, X_val, y_train, y_val = train_test_split(X , train['target'] , train_size = 0.75 , shuffle=True)

# for c in [0.01, 0.05, 0.25, 0.5, 1 ]:
#     lr = LogisticRegression(C=c)
#     lr.fit(X_train, y_train)
#     print ("Accuracy for C=%s: %s"% (c, accuracy_score(y_val, lr.predict(X_val))))

In [15]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


# Define the models to evaluate and their hyperparameter space
models = [
    {
        'model': LogisticRegression(),
        'param_grid': {
            'C': [0.1, 1.0, 10.0],
            'penalty': ['l1', 'l2']
        }
    },
    {
        'model': SVC(),
        'param_grid': {
            'C': [0.1, 1.0, 10.0],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
        }
    },
    {
        'model': RandomForestClassifier(),
        'param_grid': {
            'n_estimators': [10, 50, 100],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5]
        }
    }
]

# Evaluate each model using cross-validation with grid search over the hyperparameter space
for model in models:
    grid = GridSearchCV(model['model'], param_grid=model['param_grid'], cv=5)
    grid.fit(X, train['target'])
    print("%s: %0.2f accuracy (+/- %0.2f)" % (model['model'].__class__.__name__, grid.best_score_, grid.cv_results_['std_test_score'][grid.best_index_] * 2))
    print("Best hyperparameters: %s\n" % grid.best_params_)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression: 0.76 accuracy (+/- 0.00)
Best hyperparameters: {'C': 0.1, 'penalty': 'l2'}

SVC: 0.77 accuracy (+/- 0.00)
Best hyperparameters: {'C': 0.1, 'kernel': 'linear'}

RandomForestClassifier: 0.76 accuracy (+/- 0.00)
Best hyperparameters: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 10}

